In [56]:
import numpy as np
# Loading pickled objects
import pickle

# Latent Dirchlet Model
from gensim.models import LdaModel

# Build corpus
from gensim import corpora

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.mixture import LogisticRegression
from sklearn.model_selection import train_test_split

## Load Processed Data

In [2]:
with open('../data/processed_data.pkl', mode='rb') as file:
    processed_data = pickle.load(file)

## Load Title Tokens

In [26]:
titles = [' '.join(record['title_tokens']) for record in processed_data]
titles[0]

'railcar team entry'

## Make Title Bag of Words

In [41]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(titles)
X.shape

(9485, 2520)

## Load Description Tokens

In [5]:
description_tokens =[record['description_tokens'] for record in processed_data]
description_tokens[0]

['company_overview',
 'come_join_win',
 'team',
 'since',
 'plastic_express',
 'lead',
 'bulk',
 'trucking',
 'bulk',
 'terminal',
 'packaging',
 'warehouse',
 'need',
 'plastic',
 'industry',
 'strategic',
 'location',
 'modern',
 'system',
 'dedicate',
 'employee',
 'allow_us',
 'provide',
 'custom',
 'tailored',
 'logistical',
 'solution',
 'fulfill',
 'challenging',
 'need',
 'customer',
 'plastic_express',
 'operate',
 'warehouse',
 'location',
 'rail',
 'terminal',
 'across',
 'us',
 'many',
 'plastic_express',
 'site',
 'also',
 'handle',
 'commodity',
 'include',
 'paper',
 'roll',
 'steel',
 'building',
 'material',
 'dry',
 'bulk',
 'material',
 'plastic_express',
 'operate',
 'roughly',
 'truck',
 'approximately',
 'trailer',
 'perform',
 'full',
 'bulk',
 'truck',
 'distribution',
 'business',
 'plastic_express',
 'headquarter',
 'city',
 'industry',
 'ca',
 'employee',
 'nationwide',
 'goal',
 'always',
 'exceed',
 'customer',
 'expectation',
 'attitude',
 'differentiate_u

## Make Description Bag of Words

In [6]:
dictionary = corpora.Dictionary(description_tokens)
dictionary.filter_extremes(no_below=3)
description_corpus = [dictionary.doc2bow(token) for token in description_tokens]
description_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 4),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 3),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 3),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 2),
 (39, 1),
 (40, 1),
 (41, 3),
 (42, 1),
 (43, 2),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 2),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 2),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 2),
 (71, 2),
 (72, 1),
 (73, 1),
 (74, 2),
 (75, 3),
 (76, 1),
 (77, 2),
 (78, 4),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 2),
 (83, 1),
 (84, 1),
 (85, 2),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 3),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 1),
 (100, 1),

## Load Best Model

In [7]:
file_path = '../model/LDA-24topics'
model = LdaModel.load(file_path)

## Get Description Topic Distributions

In [39]:
rows = []
for description in description_corpus:
    topics = model.get_document_topics(description)
    vec = np.zeros(24)
    for key, prob in topics:
        vec[key] = prob
    rows.append(vec)
y = np.array(rows)

In [40]:
y.shape

(9485, 24)

In [45]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [57]:
title_model = LogisticRegression()

In [60]:
title_model.fit(X_train, y_train[:,0])

ValueError: Unknown label type: 'continuous'